In [1]:
#importing depedencies
import pandas as pd

In [2]:
#pulled canada postal code table from website into dataframe
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url)
table_df = tables[0]

In [3]:
#display first 2 columns of the postalcode df
table_df.head(2)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned


In [4]:
# chancge column name from Postcode to PostalCode
table_df = table_df.rename(columns={"Postcode":"PostalCode"})

In [5]:
# removing rows where the Borough is unassigned
table_df=table_df[table_df["Borough"]!= "Not assigned"]

In [6]:
table_df.head(3)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
# grouping by postalcode and concatinating neighbourhood comma separated.
table_df_concat = table_df.groupby(['PostalCode','Borough'], sort=False).agg( ','.join)
table_df_concat.head()

,,Neighbourhood
PostalCode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Not assigned


In [8]:
# resetting table index
table_df_concat = table_df_concat.reset_index()

In [9]:
#filtering by neighbourhood that is not assigned
no_neigh=table_df[table_df["Neighbourhood"]== "Not assigned"]
no_neigh

,PostalCode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [10]:
#populating Not assigned cell in the neighbourhood column with a value
table_df_concat.at[4, "Neighbourhood"]= "Queen's Park"

In [11]:
#viewing changes made to 
table_df_concat.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [12]:
table_df_concat.shape

(103, 3)

In [13]:
#importing lat long csv into dataframe
url2="https://cocl.us/Geospatial_data"
csv = pd.read_csv(url2)
lat_long_df = csv
# chancge column name from Postal Code to PostalCode
lat_long_df = lat_long_df.rename(columns={"Postal Code":"PostalCode"})
lat_long_df.head(3)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [14]:
# merging postal dataframes on postal code column to create new table with lat long
postalcode_df = pd.merge(lat_long_df, table_df_concat, on="PostalCode")

In [15]:
# rearranged dataframe columns
postalcode_df = postalcode_df[["PostalCode", "Borough", "Neighbourhood", "Latitude", "Longitude"]]
postalcode_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
postalcode_df = postalcode_df.drop(columns = ["PostalCode"])

In [17]:
# imported folium for map rendering 
import folium

In [18]:
#Generating initial map
map_toronto = folium.Map(location=[43.651070, -79.347015], tiles='Stamen Terrain', zoom_start=10)
map_toronto

# adding markers to map of tornoto Canada
for lat, lng, borough, neighborhood in zip(postalcode_df['Latitude'], \
                                           postalcode_df['Longitude'], \
                                           postalcode_df['Borough'], \
                                           postalcode_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
#importing client ID and client secret from config file. used this method so i can use git ignore to protect api key
from config import client_id, client_secret

In [20]:
# creating API variables
CLIENT_ID = client_id 
CLIENT_SECRET = client_secret 
VERSION = '20180605' 

In [21]:
#importing required libraries
from sklearn.cluster import KMeans
import numpy as np
import requests

In [22]:
#the function below pulls the top 100 venues within a 500m radius of toronto neighborhoods
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
#calling the function to get nearby venues
toronto_venues = getNearbyVenues(names=postalcode_df['Neighbourhood'],
                                   latitudes=postalcode_df['Latitude'],
                                   longitudes=postalcode_df['Longitude']
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [ ]:
#viewing the df 
print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
#getting an idea of how many venues that were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count().sort_values(by="Neighborhood", ascending=False).head()

In [ ]:
unique_categories = len(toronto_venues['Venue Category'].unique())
print(f'There are {unique_categories} uniques categories')

In [ ]:
# one hot encoding to convert strings to numerical values
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

#move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()
# toronto_venues.head()

In [ ]:
toronto_grouped.shape

<h5>Displaying neighbourhoods and their top 5 venues</h5>

In [ ]:
num_top_venues = 5

for hood in (toronto_grouped['Neighborhood']):
    print(f"----{hood}----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster label

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = postalcode_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

In [ ]:
# create map
tor_map_clusters = folium.Map(location=[43.651070, -79.347015], tiles='Stamen Terrain', zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(postalcode_df['Latitude'], postalcode_df['Longitude'], postalcode_df['Neighborhood'], postalcode_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(tor_map_clusters)
       
tor_map_clusters